<a href="https://colab.research.google.com/github/Matthew12o/StakeSlashSwap/blob/main/Parachain_Offering_Token_Issuance_Calculator_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parachain Slot Token Calculator

## Assumptions
The following calculations are made with an assumption that the expected return from a receiving parachain token must be greater than or equal to the cost to participate in the offering

\begin{equation*}
\text{Token Value}_t \geq \text{Participation Cost}
\end{equation*}

\begin{equation*}
\text{Token Value}_t\geq -\Delta \text{Price of DOT} + \text{Opportunity Cost} + \text{Cost Of Capital} 
\end{equation*}

### Where

Generalized Asset Price

\begin{equation*}
P_t = P_0 e^{\sigma_P \beta \left( t \right) + \mu t} 
\end{equation*}

and 


\begin{equation*}
N_C = \text{Number of Parachain Tokens} \\
N_D = \text{Number of DOT Required For a Parachain Slot} \\
C_0 = \text{Price of Parachain Token at time 0} \\ 
C_t = \text{Price of Parachain Token at time t} \\
D_0 = \text{Price of DOT at time 0} \\
D_t = \text{Price of DOT at time t} \\
\sigma_X = \text{Volatility of Asset X} \\
\beta(x) = \text{Wiener Process} \\
r_s = \text{Average DOT Staking Reward} \\
r_c = \text{Cost of Capital for Base Currency} \\
\mu_X = \text{Mean Return of Asset X}
\end{equation*}

### Therefore Generalized Token Economics will be
\begin{equation*}
N_{C}C_t \geq N_{D} \left(D_0 - D_t + D_0 \left(e^{r_st} - 1 \right) + D_0 \left(e^{r_ct} - 1 \right) \right)
\end{equation*}

\begin{equation*}
N_{C}C_0 e^{\sigma_C \beta \left( t \right) + \mu_C t} \geq N_{D} D_0 \left( -e^{\sigma_D \beta \left( t \right) + \mu_D t} + e^{r_st} + e^{r_ct} - 1 \right) 
\end{equation*}

### Expected Value of Return is
\begin{equation*}
E[x] = N_{C}C_0 e^{\mu_C t} \geq N_{D} D_0 \left( -e^{\mu_D t} + e^{r_st} + e^{r_ct} - 1 \right)
\end{equation*}
or 
\begin{equation*}
E[x] = \frac{N_{C}C_0}{N_{D}D_0} \geq \frac{-e^{\mu_D t} + e^{r_st} + e^{r_ct} - 1}{e^{\mu_C t}}
\end{equation*}

In [ ]:
#@title
%matplotlib inline
from IPython.display import display
from ipywidgets import *
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import math
from matplotlib import pyplot as plt

import numpy as np

# mean return
mu_d = widgets.FloatSlider(value = 0.00, step = 0.0025, min = -1, max = 15, readout_format = '.2%')
mu_c = widgets.FloatSlider(value = 0.00, step = 0.0025, min = -1, max = 15, readout_format = '.2%')
# interest rates
r_s = widgets.FloatSlider(value = 0.10, step = 0.001, min = 0.025, max = 1, readout_format = '.2%')
r_c = widgets.FloatSlider(value = 0.005, step = 0.0001, min = -0.25, max = 0.25, readout_format = '.2%')
# Number of Tokens
N_D = widgets.IntSlider(value = 2000000, min = 1, max = 10000000, readout_format=',')
N_C = widgets.FloatSlider(value = 1, min = 0, max = 100, step = 0.001, readout_format='$.3f')
t = widgets.IntSlider(value = 2, min = 1, max = 4)
#Price of DOT
D_0 = widgets.FloatSlider(value = 5, min = 0, max = 10, step = 0.01, readout_format='$.2f')
T_0 = widgets.FloatSlider(value = 1, min = 0, max = 100, step = 0.001, readout_format='.3f')

def getCostPerStakingRate():
  x = np.linspace(start=0.025, stop=1, num=100)
  y = []
  for i in x:
    y.append(Cost(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, i, r_c.value, t.value))
  return x, y

def getCostPerExpectedReturn():
  x = np.linspace(start=-1, stop=15, num= 100)
  y = []
  for i in x:
    y.append(Cost(N_C.value, N_D.value, D_0.value, mu_d.value, i, r_s.value, r_c.value, t.value))
  return x, y

def CoinPrice(N_C, N_D, D_0, mu_d, mu_c, r_s, r_c, t):
    c = N_D * D_0 / N_C * (-math.exp(mu_d * t / 2) + math.exp(r_s * t/2) + math.exp(r_c * t/2) - 1)/ math.exp(mu_c * t / 2)  
    cost = (-math.exp(mu_d * t / 2) + math.exp(r_s * t/2) + math.exp(r_c * t/2) - 1)/ math.exp(mu_c * t / 2) * 2 / t
    print('Martingale Annualized Parachain Cost: {:,.2%}'.format(cost))
    print('Value of Parachain Slot: ${:,.2f}'.format(N_D * D_0))
    print('Value of Tokens Issued: ${:,.2f}'.format(c * N_C))
    print('Price of Tokens at Issuance: ${:,.2f}'.format(N_C))
    print('Martingale Token Issuance: {:,.0f}'.format(c))
    UpdateGraph()

def Cost(N_C, N_D, D_0, mu_d, mu_c, r_s, r_c, t):
  cost = (-math.exp(mu_d * t / 2) + math.exp(r_s * t/2) + math.exp(r_c * t/2) - 1) / math.exp(mu_c * t/2) * 2 / t
  return cost

def Issuance(N_C, N_D, D_0, mu_d, mu_c, r_s, r_c, t):
  issuance = N_D * D_0 / N_C * (-math.exp(mu_d * t / 2) + math.exp(r_s * t/2) + math.exp(r_c * t/2) - 1)/ math.exp(mu_c * t / 2)  
  return issuance

def getIssuancePerStakingRate():
  x = np.linspace(start=0.025, stop=1, num=100)
  y = []
  for i in x:
    y.append(Issuance(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, i, r_c.value, t.value))
  return x, y
def getIssuancePerExpectedReturn():
  x = np.linspace(start=-1, stop=15, num= 100)
  y = []
  for i in x:
    y.append(Issuance(N_C.value, N_D.value, D_0.value, mu_d.value, i, r_s.value, r_c.value, t.value))
  return x, y


disp_N_C = widgets.HBox([widgets.Label('Price of Token at Issuance: '), N_C])
disp_N_D = widgets.HBox([widgets.Label('Number of DOTs Required for Parachain Slot: '), N_D])
disp_D_0 = widgets.HBox([widgets.Label('Price of DOT at the time of Initial Parachain Offering: '), D_0])
disp_mu_d = widgets.HBox([widgets.Label('Mean Annual Return of DOT During the Lock Up: '), mu_d])
disp_mu_c = widgets.HBox([widgets.Label('Mean Annual Return of Tokens During the Lock Up: '), mu_c])
disp_r_s = widgets.HBox([widgets.Label('Staking Reward Rate: '), r_s])
disp_r_c = widgets.HBox([widgets.Label('Cost of Capital for the Base Currency: '), r_c])
disp_t = widgets.HBox([widgets.Label('Parachain Lock Up Periods: '), t])
disp_T_0 = widgets.HBox([widgets.Label('Price of Token at Issuance: '), T_0])

displayBox = widgets.VBox([widgets.Label('Inputs:'),disp_N_C, disp_N_D, disp_D_0, disp_mu_d, disp_mu_c, disp_r_s, disp_r_c, disp_t, widgets.Label('Outputs:')])


out = widgets.interactive_output(CoinPrice, {
    'N_C' : N_C,
    'N_D' : N_D,
    'D_0' : D_0, 
    'mu_d' : mu_d,
    'mu_c' : mu_c,
    'r_s' : r_s,
    'r_c' : r_c,
    't' : t
})

def UpdateGraph():
  x_g1, y_g1 = getCostPerStakingRate()
  x_g2, y_g2 = getCostPerExpectedReturn()
  x_g3, y_g3 = getIssuancePerStakingRate()
  x_g4, y_g4 = getIssuancePerExpectedReturn()

  fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2)
  fig.set_figheight(9)
  fig.set_figwidth(27)
  fig.subplots_adjust(hspace=0.25, wspace=0.25)

  ax1.set_title('Staking Reward Rate v. Annualized Parachain Cost')
  ax1.set(xlabel='Staking Reward Rate', ylabel='Annual Parachain Cost')
  ax1.plot(x_g1, y_g1)
  ax1.plot(r_s.value, Cost(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, r_s.value, r_c.value, t.value), 'rD')
 
  y_value=['{:.0%}'.format(x) for x in ax1.get_yticks()]
  ax1.set_yticklabels(y_value)

  x_value = ['{:.0%}'.format(x) for x in ax1.get_xticks()]
  ax1.set_xticklabels(x_value)

  ax2.set_title('Expected Return on Token v. Annaulized Parachain Cost')
  ax2.set(xlabel='Expected Return on Token', ylabel='Annual Parachain Cost')
  ax2.plot(x_g2, y_g2)
  ax2.plot(mu_c.value, Cost(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, r_s.value, r_c.value, t.value), 'rD')

  y_value=['{:.0%}'.format(x) for x in ax2.get_yticks()]
  ax2.set_yticklabels(y_value)

  x_value = ['{:.0%}'.format(x) for x in ax2.get_xticks()]
  ax2.set_xticklabels(x_value)

  ax3.set_title('Staking Reward Rate v. Total Issuance for {} Periods of Parachain Slot'.format(t.value))
  ax3.set(xlabel='Staking Reward Rate', ylabel='Token Issuance')
  ax3.plot(x_g3, y_g3)
  ax3.plot(r_s.value, Issuance(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, r_s.value, r_c.value, t.value), 'rD')

  y_value=['{:,.0f}'.format(x) for x in ax3.get_yticks()]
  ax3.set_yticklabels(y_value)

  x_value = ['{:.0%}'.format(x) for x in ax3.get_xticks()]
  ax3.set_xticklabels(x_value)

  ax4.set_title('Expected Return on Token v. Total Issuance for {} Periods of Parachain Slot'.format(t.value))
  ax4.set(xlabel='Expected Return on Token', ylabel='Token Issuance')
  ax4.plot(x_g4, y_g4)
  ax4.plot(mu_c.value, Issuance(N_C.value, N_D.value, D_0.value, mu_d.value, mu_c.value, r_s.value, r_c.value, t.value), 'rD')

  y_value=['{:,.0f}'.format(x) for x in ax4.get_yticks()]
  ax4.set_yticklabels(y_value)

  x_value = ['{:.0%}'.format(x) for x in ax4.get_xticks()]
  ax4.set_xticklabels(x_value)

  fig.show()

display(displayBox, out)


Output()